# Deep CNN
Here we can:
* Retrain the entire network with randomly initialized weights
* Train network with pre-trained encoder weights and randomly initialized FC layer weights(fine-tuning)
Reconstruction of images plotted at the end of notebook.

In [2]:
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Dropout, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from math import ceil
from random import randint
from shared.autoencoderHelpers import read_n_images, generate_img_from_folder, get_input_shape, get_num_examples, plot_history, get_images, bgr2rgb, plot_reconstruction

## Generate model(s)
* Encoder
* FFNN

**Set the layers for auto-encoder**

In [3]:
DATA_DIR = '../data'

In [4]:
autoencoder = tf.keras.models.load_model('../models/autoencoder/config2/model') 

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [5]:
autoencoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     (None, 200, 200, 3)       0         
_________________________________________________________________
enc_conv1 (Conv2D)           (None, 200, 200, 16)      448       
_________________________________________________________________
enc_max_pool1 (MaxPooling2D) (None, 100, 100, 16)      0         
_________________________________________________________________
enc_conv2 (Conv2D)           (None, 100, 100, 8)       1160      
_________________________________________________________________
enc_max_pool2 (MaxPooling2D) (None, 50, 50, 8)         0         
_________________________________________________________________
enc_conv3 (Conv2D)           (None, 50, 50, 8)         584       
_________________________________________________________________
enc_max_pool3 (MaxPooling2D) (None, 25, 25, 8)         0         
__________

<img src="../CNN_schematic.png"  width= 30% align = right>

## Visual representation


Visual representation of what we are doing in Keras essentially.

We train the models in 3 distinct ways:
* Use model as a whole (CNN, green)
    * Initialize the weights randomly and train the whole model
    * Initialize the encoder layer weights as they are and train the whole model (fine tuning)
* Use model as an encoder (red) and classifier (blue)
    * Keep the encoder layer with the trained weights and initialize the classifier model weights randomly and train **only** the classifier model.

In [6]:
num_classes = get_num_classes(DATA_DIR)

layer_names = [autoencoder.layers[i].name for i in range(7)]
print("Layers of encoder:\n{}".format(layer_names))

in_clf_model = Input(shape=autoencoder.layers[6].output_shape[1:], name='in_classifier')
encoder_out = autoencoder.layers[6].output

flat = Flatten()(in_clf_model)
x = Dense(2048, activation=tf.nn.relu, name='fc_1')(flat)
x = Dense(1024, activation=tf.nn.relu, name='fc_2')(x)
x = Dropout(rate=0.4, name='dropout')(x)
output_full_model = Dense(num_classes, activation=tf.nn.softmax, name='output_layer')(x)


encoder = Model(inputs=autoencoder.inputs, outputs=autoencoder.layers[6].output)
clf_model = Model(inputs=in_clf_model, outputs=output_full_model)
#full_model = Model(inputs=autoencoder.inputs, outputs=output_full_model) # Full model not implemented in this notebook

Layers of encoder:
['input_layer', 'enc_conv1', 'enc_max_pool1', 'enc_conv2', 'enc_max_pool2', 'enc_conv3', 'enc_max_pool3']
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [8]:
encoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     (None, 200, 200, 3)       0         
_________________________________________________________________
enc_conv1 (Conv2D)           (None, 200, 200, 16)      448       
_________________________________________________________________
enc_max_pool1 (MaxPooling2D) (None, 100, 100, 16)      0         
_________________________________________________________________
enc_conv2 (Conv2D)           (None, 100, 100, 8)       1160      
_________________________________________________________________
enc_max_pool2 (MaxPooling2D) (None, 50, 50, 8)         0         
_________________________________________________________________
enc_conv3 (Conv2D)           (None, 50, 50, 8)         584       
_________________________________________________________________
enc_max_pool3 (MaxPooling2D) (None, 25, 25, 8)         0         
Total para

In [7]:
clf_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
in_classifier (InputLayer)   (None, 25, 25, 8)         0         
_________________________________________________________________
flatten (Flatten)            (None, 5000)              0         
_________________________________________________________________
fc_1 (Dense)                 (None, 2048)              10242048  
_________________________________________________________________
fc_2 (Dense)                 (None, 1024)              2098176   
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
output_layer (Dense)         (None, 5)                 5125      
Total params: 12,345,349
Trainable params: 12,345,349
Non-trainable params: 0
________________________________________________________________

## Compile the model and get model summaries
Compiling only happens for `clf_model` model (as we are not training the encoder).

In [9]:
clf_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

## Training the classifier
* Tune for batch size, number of epochs, etc.

In [69]:
import pickle
import glob
import cv2

def get_num_classes(data_dir):
    """
    Get the number of classes.
    :param data_dir: str - data directory
    :return: int - number of classes
    """
    mode = 'training' # arbitrary
    loc = "{}/{}".format(data_dir, mode)
    with open('{}/labels.pickle'.format(data_dir), 'rb') as f:
        data = pickle.load(f)

    modes = list(data.keys())

    assert glob.glob(data_dir), "Check directory."
    assert glob.glob("{}/*.jpg".format(loc)), "Check file extension (should be 'jpg')."
    i = 0  # Arbitrarily chosen
    return len(data[modes[i]][i][-1])

def generate_enc_from_folder(data_dir, mode, batch_size, encoder):
    """

    """
    loc = "{}/{}".format(data_dir, mode)
    while True:
        with open('{}/labels.pickle'.format(data_dir), 'rb') as f:
            data = pickle.load(f)

        modes = list(data.keys())
        del modes[-1]

        assert mode in modes, "'{}' not a valid mode (must be one of {})".format(mode, str(modes))
        assert glob.glob(loc), "Check directory."
        assert glob.glob("{}/*.jpg".format(loc)), "Check file extension (should be 'jpg')."
        for idx in range(0, len(data[mode]), batch_size):
            start = idx
            end = idx + batch_size
            images, labels = read_n_images(data[mode], start, end, loc)
            enc_images = encoder.predict(images/255, batch_size=batch_size, steps=1)
            yield (enc_images, labels)

In [70]:
labels

array([2, 4, 4, 1, 1, 2, 1, 1, 1, 4])

In [71]:
batchSize = 8
epochs = 30

BATCH_SIZE_TRAIN = 10
NUM_SAMPLES_TRAIN = get_num_examples(DATA_DIR, 'training')
STEPS_PER_EPOCH = ceil(NUM_SAMPLES_TRAIN/BATCH_SIZE_TRAIN)

BATCH_SIZE_VAL= 10
NUM_SAMPLES_VAL = get_num_examples(DATA_DIR, 'validation')
VALIDATION_STEPS=ceil(NUM_SAMPLES_VAL/BATCH_SIZE_VAL)

EPOCHS = 2

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
history = clf_model.fit_generator(generate_enc_from_folder(DATA_DIR, 'training', BATCH_SIZE_TRAIN, encoder=encoder),# shuffle=True,
                                    validation_data=generate_enc_from_folder(DATA_DIR, 'validation', BATCH_SIZE_VAL, encoder=encoder),
                                    steps_per_epoch=STEPS_PER_EPOCH, validation_steps=VALIDATION_STEPS,
                                    epochs=EPOCHS)


Epoch 1/2


InvalidArgumentError: Tensor input_layer:0, specified in either feed_devices or fetch_devices was not found in the Graph

In [43]:
generate_enc_from_folder(DATA_DIR, 'training', BATCH_SIZE_TRAIN, encoder=encoder)[-1]

IN


array([2, 4, 4, 1, 1, 2, 1, 1, 1, 4])

## Accessment of model perfomance
* Visualize the training history
* Generate predictions for auto-encoder (reconstructed images)
* Visualize original and reconstructed images (for comparison)

In [ ]:
plot_history(history)

In [ ]:
#Decoding images
BATCH_SIZE_TEST = 10
NUM_SAMPLES_TEST = get_num_examples(DATA_DIR, 'test')
STEPS_TEST = ceil(NUM_SAMPLES_TEST/BATCH_SIZE_TEST)

decoded_imgs = autoencoder.predict_generator(generate_img_from_folder(DATA_DIR, 'test', BATCH_SIZE_TEST),steps=STEPS_TEST)

In [ ]:
num_images = 5
idxs = [randint(0, NUM_SAMPLES_TEST) for i in range(num_images)]

plot_reconstruction(decoded_imgs, DATA_DIR, 'test', idxs)

**Encoded representation is by a `25x25x8` array so it is not implemented for now. One alternative is to display each layer as a grayscale image.**

## Sanity check:
See if  encoder-decoder are consisted with entire auto-encoder architecture

In [ ]:
encoded_imgs = encoder.predict_generator(generate_img_from_folder(DATA_DIR, 'test', BATCH_SIZE_TEST),steps=STEPS_TEST)
decoded_imgs1 = decoder.predict(encoded_imgs)
if np.all(decoded_imgs1 == decoded_imgs):
    print("Output from decoded model is the same as the output from auto-encoder (everything looks good).")